<a href="https://colab.research.google.com/github/kellerpires/live/blob/master/bolsavalores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pyspark

In [20]:
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
    .builder \
    .appName("trabalho pratico 1 - Cientista de Dados - Apache Spark") \
    .getOrCreate()

spark.version

'3.5.0'

In [31]:
# Carregando a planilha CSV
bolsavalores_df = spark.read.csv('/content/all_stocks_5yr.csv',header='True',inferSchema='True')

# Detalhes dos atributos em https://www.kaggle.com/c/titanic/data?select=train.csv
bolsavalores_df.printSchema()

root
 |-- date: date (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



Pergunta 1
Quantos registros há na planilha?

In [47]:
bolsavalores_df.count()

619040

In [48]:
bolsavalores_df.show()

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 12.7|13.26| 9419000| AAL|
|2013-02-27|13.28|13.62|13.18|13.41| 7390500| AAL|
|2013-02-28|13.49|13.63|13.39|13.43| 6143600| AAL|
|2013-03-01|13.37|13.95|13.32|13.61| 7376800| AAL|
|2013-03-04| 13.5|14.07|13.47| 13.9| 8174800| AAL|
|2013-03-05|14.01|14.05|13.71|1

Pergunta 2
Quantos registros há na planilha para a ação da Apple (AAPL)?

In [49]:
bolsavalores_df.where(bolsavalores_df.Name == 'AAPL').count()

1259

Pergunta 3
Quantas empresas distintas têm registros nessa planilha?

In [50]:
from pyspark.sql import functions as f
bolsavalores_df.select(f.countDistinct(bolsavalores_df.Name)).show()

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|                 505|
+--------------------+



In [51]:
# Importe os módulos necessários
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length


Pergunta 4
Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?
Grupo de escolhas da pergunta

In [76]:
# Basicamente eu removi os nulos, pq tem open com valor vazio, aí fiz a compração pra gerar uma nova coluna booleana,
# Daí como ele quer saber a frequência que o evento ocorre contei o total de positivos que seriam as ocorrências.
# E por fim dividi pelo total de registros da tabela

In [74]:
result = round((bolsavalores_df.select('open', 'close').filter(bolsavalores_df.open.isNotNull()).\
    withColumn('greather', bolsavalores_df.close > bolsavalores_df.open).where('greather')\
    .count()
   / bolsavalores_df.filter(bolsavalores_df.open.isNotNull()).count()) * 100, 2)

In [75]:
print('{}%'.format(result))

51.53%


In [54]:
# bolsavalores_df.select('Name').distinct().orderBy(bolsavalores_df.Name.asc()).collect()

In [55]:
bolsavalores_df.select('volume', 'date').summary().show()

+-------+-----------------+
|summary|           volume|
+-------+-----------------+
|  count|           619040|
|   mean|4321823.395568945|
| stddev|8693609.511967659|
|    min|                0|
|    25%|          1070186|
|    50%|          2081797|
|    75%|          4283455|
|    max|        618237630|
+-------+-----------------+



In [67]:
# Pergunta 10: Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento? (fazer comparação, operador booleano de igualdade)

In [68]:
result = round((bolsavalores_df.select('high', 'close').filter(bolsavalores_df.high.isNotNull()).\
    withColumn('greather', bolsavalores_df.close == bolsavalores_df.high).where('greather')\
    .count()
   / bolsavalores_df.filter(bolsavalores_df.high.isNotNull()).count()) * 100, 2)

In [70]:
print('{}%'.format(result))

1.2%


In [103]:
# Pergunta 11: Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?